# Train My First Machine Learning Model

I'm always interested in machine learning since last year I learned the concept. After watched a lot of videos and read a bunch of posts, recently I trained my first machine learning model and it's really exciting.

## Find a problem

The first also the most important thing to train a model is to find a problem which is currently handled by human but could be solved by machine. If you are one of the human it will be great because you will be willing to save yourself a lot of time.

I use [EMS](http://ems.com.cn)(Express Mail Service) a lot after my baby was born because I need to query milk powder from Germany(*Big thanks to my best friend in Germany*) and commodities from Amazon Japan. The stupid website asks every user to recognize a captcha before show their mail information. Thus I found my machine learning problem which is to recognize the captcha from their website. The captcha looks like this: 

![EMS captcha](unknow.jpg)

To our human being it's an easy task. Believe it or not it's also a piece of cake to a computer thanks to machine learning.

## Analysis the problem

The captcha is always composed by six numbers and our goal is to recognize all the numbers from a captcha image which is a typical image classification problem. There is a question what's the categories? Categories are the whole possible result to a specific image classification problem. It seems reasonable to label them from 000000 to 999999. But we will need huge amount of traning data if we categories like this. Instead we chose to cut the captcha into six pieces which contain their own number. So our category will be 0 to 9. Pretty simple! We only need to recognize each piece of the captcha and combine their result together to get final result.

We need to write a simple script to cut the captcha image. 

In [1]:
from PIL import Image
import random

name = "unknow.jpg"

y = 0
w = 13
h = 20

x = 5

im = Image.open(name)
for i in range(6):
    x += w if i > 0 else 0 
    region = im.crop((x, y, x + w, y + h))
    resize = region.resize((128, 197), Image.BILINEAR)
    segmentName = random.randint(1, 9999)
    resize.save("%d.jpeg" % segmentName)

## Prepare training data

Our machine learning model needs to be fed by tranining data. How can we get data? It's always been a difficult problem for machine learning newbies. I use a most stupid method. Download a lot of captcha from their website and use their value for name. These captcha named with their content are the origin data. We need to preprocess those captcha and organize them correctly before we do training. 

We need to create ten directories with name from 0 to 9 and put each segment number cut from origin captcha into their corresponding directory. Here is the codes: 

In [ ]:
from PIL import Image
import random
import os

imageDir = "./digit_images"
if not os.path.exists(imageDir):
    os.makedirs(imageDir)

for i in range(10):
    dir = "./%s/%d" % (imageDir, i)
    if not os.path.exists(dir):
        os.makedirs(dir)

dataDir = "./captcha/"
y = 0
w = 13
h = 20

for name in os.listdir(dataDir):
    x = 5
    value = name.split(".")[0]
    im = Image.open(dataDir + name)
    for i in range(6):
        x += w if i > 0 else 0 
        region = im.crop((x, y, x + w, y + h))
        resize = region.resize((128, 197), Image.BILINEAR)
        segmentName = random.randint(1, 9999)
        resize.save("./%s/%s/%d.jpeg" % (imageDir, value[i], segmentName))

## Train model

With training data prepared we can train our model simply with a script.

In [ ]:
python -m scripts.retrain \
  --bottleneck_dir=tf_files/bottlenecks \
  --how_many_training_steps=500 \
  --model_dir=tf_files/models/ \
  --summaries_dir=tf_files/training_summaries/mobilenet_0.50_128 \
  --output_graph=tf_files/retrained_graph.pb \
  --output_labels=tf_files/retrained_labels.txt \
  --architecture=mobilenet_0.50_128 \
  --image_dir=tf_files/flower_photos

If you want to learn more about the script above please check out [4. (Re)training the network](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/#3) from TensorFlow For Poets. The whole traning process should cost you dozens of seconds. 

# Use model to predict

After training we could use our model to predict a segment number.

In [15]:
%run -i 'scripts/label_image.py --image=digit_images/0/155.jpeg'

ERROR:root:File `'scripts/label_image.py --image=digit_images/0/155.jpeg.py'` not found.
